## Расчет расстояния между геокоординатами по прямой в Python

Считаем расстояния между объектами по их координатам. В данном примере будем считать расстояния от населенных пунктов до школ и выбирать ближайшую школу. 

Важно: расстояния считаются по прямой, без учета дорог, гор, рек и пр., поэтому дают только общее представление о дальности объектов. Чтобы точно рассчитать дистанцию и время в пути по дорогам, можно воспользоваться API матрицы расстояний Яндекс.Карт (платно, либо можно попробовать попросить бесплатно в исследовательских и научных целях по индивидуальному запросу) или OSRM — Open Sourced Routing Machine (бесплатно)

### 1. Скачивание данных

1. Данные по населенным пунктам скачиваем на сайте ИНИД (Инфраструктура научно-исследовательских данных) https://www.data-in.ru/data-catalog/datasets/160/
2. Данные по школам — парсингом сайта https://schoolotzyv.ru/schools/9-russia/ (данные доступны в формате json по ссылке https://drive.google.com/file/d/12zw_ZroDTfYKSjsUCPZy3GRhTluzmljx/view?usp=sharing) По некоторым регионам на нем есть не все школы. Можно дополнить данными с сайтов https://arhangelsk.fulledu.ru/, https://russiaedu.ru/, набора открытых данных с лицензиями Рособрнадзора (требуется парсинг xml) http://obrnadzor.gov.ru/otkrytoe-pravitelstvo/opendata/7701537808-fbdrl/ и других источников.

### 2. Загрузка библиотек

In [2]:
import pandas as pd # будем работать в pandas
import numpy as np # numpy и sklearn для математических расчетов и использования формул
import sklearn.neighbors
import json # для работы с файлами в формате json

### 3. Загрузка и подготовка данных

In [8]:
# загружаем датасет со школами в pandas

with open('/Users/me/Downloads/schoolotzyv (1).json', 'r') as read_file:
    otzyv = json.load(read_file)

In [83]:
otzyv = pd.DataFrame(otzyv)
otzyv

,geo_lat,geo_long,url,text
0,54.154966,155.843419,https://schoolotzyv.ru/component/schools/9-rus...,Школа с. Устьевое
1,52.928782,156.573372,https://schoolotzyv.ru/component/schools/9-rus...,Большерецкая школа №5
2,52.670512,156.238983,https://schoolotzyv.ru/component/schools/9-rus...,Школа №1 п. Октябрьский
3,52.824094,156.285057,https://schoolotzyv.ru/component/schools/9-rus...,Школа с. Усть-Большерецк
4,51.494019,156.501408,https://schoolotzyv.ru/component/schools/9-rus...,Школа п. Озерновский
...,...,...,...,...
50804,55.924598,158.693755,https://schoolotzyv.ru/component/schools/9-rus...,Школа с. Эссо
50805,56.053578,158.971110,https://schoolotzyv.ru/component/schools/9-rus...,Школа с. Анавгай
50806,54.299330,155.948234,https://schoolotzyv.ru/component/schools/9-rus...,Школа с. Соболево
50807,55.026700,155.591163,https://schoolotzyv.ru/component/schools/9-rus...,Школа п. Крутогоровский


In [84]:
# нам нужно узнать регионы, для этого выбираем любую ссылку, переходим по ней и смотрим, как в ссылках зашит нужный нам регион

otzyv.loc[1,'url']

'https://schoolotzyv.ru/component/schools/9-russia/131-kamchatskij/64530-obshheobrazovatelnaya-shkola'

In [85]:
# для примера возьмем Нижегородскую область (если взять всю Россию, считаться будет долго)
# она в ссылках на школы помечена как "146-nizhegorodskaya"
# выбираем только те строки, которые содержат такую запись
# подробнее о том как фильтровать датафреймы в pandas — в уроке https://istories.media/workshops/2021/03/05/python-biblioteka-pandas-chast-1/


schools_nn = otzyv[otzyv['url'].str.contains('146-nizhegorodskaya')]
len(schools_nn)

1245

In [65]:
schools_nn

,geo_lat,geo_long,url,text
7598,55.784521,44.781727,https://schoolotzyv.ru/component/schools/9-rus...,Большемурашкинская школа
7599,55.825548,44.761182,https://schoolotzyv.ru/component/schools/9-rus...,Холязинская школа
7600,55.886998,44.695407,https://schoolotzyv.ru/component/schools/9-rus...,Кишкинская школа
7601,55.820749,44.688356,https://schoolotzyv.ru/component/schools/9-rus...,Советская школа
7602,55.829173,44.525374,https://schoolotzyv.ru/component/schools/9-rus...,Карабатовская школа
...,...,...,...,...
8838,56.181499,44.075750,https://schoolotzyv.ru/component/schools/9-rus...,Мореновская санаторно-лесная школа
8839,56.288776,44.003723,https://schoolotzyv.ru/component/schools/9-rus...,Нижегородская школа-интернат для слепых
8840,56.530916,43.459353,https://schoolotzyv.ru/component/schools/9-rus...,Нижегородская кадетская школа-интернат им. Мар...
43394,56.263447,44.015293,https://schoolotzyv.ru/component/schools/9-rus...,Школа №131 Нижний Новгород


In [86]:
# поскольку мы выбрали только некоторые строки, индексы датафрейма сбились
# чтобы они опять начинались с нуля, нужно их сбросить методом reset_index

schools_nn = schools_nn.reset_index(drop = True)

In [66]:
# загружаем датасет с населенными пунктами

places = pd.read_csv('/Users/me/Downloads/places.csv')
places

,id,region,municipality,settlement,type,population,children,latitude_dms,longitude_dms,latitude_dd,longitude_dd,oktmo,dadata,rosstat
0,0,Орловская область,Болховский,Колонтаева,д,0,0,53.22.07,035.54.36,53.368611,35.910000,5.460442e+10,0,1
1,1,Республика Крым,Алушта,Пушкино,с,273,0,44.35.45,034.20.27,44.595833,34.340833,3.570300e+10,0,1
2,2,Липецкая область,Лев-Толстовский район,Барятино,с,7,1,53.15.46,039.30.14,53.262778,39.503889,4.263641e+10,0,1
3,3,Тверская область,Селижаровский район,Хилово,д,2,0,56.54.20,033.25.09,56.905556,33.419167,2.865043e+10,0,1
4,4,Томская область,Парабельский район,Басмасово,д,6,0,58.38.12,082.02.40,58.636667,82.044444,6.964444e+10,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155916,155916,Ульяновская область,Вешкаймский район,Ребровка,д,0,0,53.59.12,046.48.35,53.986667,46.809722,7.360744e+10,0,1
155917,155917,Псковская область,Палкинский район,Шадрица,д,1,0,57.29.03,028.08.48,57.484167,28.146667,5.863742e+10,0,1
155918,155918,Псковская область,Себежский район,Овчинниково,д,0,0,56.19.48,028.26.17,56.330000,28.438056,5.865410e+10,0,1
155919,155919,Тульская область,рабочий пос. Новогуровский,Новогуровский,рп,3398,305,54.28.03,037.20.19,54.467500,37.338611,7.070200e+10,1,0


In [78]:
# для примера выбираем только Нижегородскую область и только один район — например, Воскресенский 
# (чтобы точно всё быстро посчиталось на небольшом объеме данных)

places_voskresensk = places[(places['region'] == 'Нижегородская область') & 
                            (places['municipality'] == 'Воскресенский район')]
len(places_voskresensk)

163

In [79]:
# опять обновляем индексы, чтобы шли с нуля 

places_voskresensk = places_voskresensk.reset_index(drop = True)

### 4. Создание словарей

Прежде чем считать расстояния, нам понадобится создать два словаря — с населенными пунктами и со школами. Это нужно, чтобы в дальнейшем мы могли сопоставить разные датафреймы по какому-то уникальному параметру, доставать и добавлять нужные нам данные. Это нам пригодится на последних этапах. 

Для населенных пунктов таким уникальным параметром будет код ОКТМО (Общероссийского классификатора территорий муниципальных образований)

Перед этим нам нужно убедиться, что:
1) Во всех нужных нам столбцахуказан правильный тип данных

2) Удалить дубликаты в тех столбцах, которые будут уникальным ключом для словаря (для населенных пунктов это поле oktmo, для школ — url, ссылка)

In [80]:
# смотрим какой тип данный в столбце 'oktmo'

places_voskresensk['oktmo'].dtype

dtype('float64')

In [81]:
# float, а нам нужны строки, поэтому меняем сначала на integer, а затем на str

In [17]:
places_voskresensk['oktmo'] = places_voskresensk['oktmo'].astype('int').astype('str')
places_voskresensk['oktmo'].dtype

dtype('O')

In [73]:
# удаляем дубликты и сбрасываем индексы 

places_voskresensk[places_voskresensk['oktmo'].duplicated()]

,id,region,municipality,settlement,type,population,children,latitude_dms,longitude_dms,latitude_dd,longitude_dd,oktmo,dadata,rosstat
94,88013,Нижегородская область,Воскресенский район,Безводное,д,17,0,56.56.48,045.25.17,56.946667,45.421389,2.262245e+10,0,1
107,105267,Нижегородская область,Воскресенский район,Осиновка,д,57,8,56.37.21,045.33.42,56.622500,45.561667,2.262242e+10,0,1
118,116684,Нижегородская область,Воскресенский район,Пузеево,д,34,3,56.49.46,045.14.38,56.829444,45.243889,2.262240e+10,0,1
133,127088,Нижегородская область,Воскресенский район,Богданово,д,32,10,56.44.03,045.38.21,56.734167,45.639167,2.262241e+10,0,1
135,129089,Нижегородская область,Воскресенский район,Калиниха,д,233,68,56.48.50,045.25.58,56.813889,45.432778,2.262215e+10,0,1


In [29]:
places_voskresensk = places_voskresensk[~places_voskresensk['oktmo'].duplicated()]
places_voskresensk = places_voskresensk.reset_index(drop = True)

In [30]:
# собираем датафрейм с населенными пунктами в словарь 


places_by_oktmo = {} # пустой словарь 
for i in range(len(places_voskresensk)): # i это индекс, порядковый номер строки
    el = places_voskresensk.loc[i] # записываем в переменную el строку с индексом i 
    places_by_oktmo[el['oktmo']] = el # создаем словарь, где ключом будет код октмо, 
                                    # а значением — соответствующая ему строка датафрейма
len(places_voskresensk)


158

In [ ]:
# (чтобы посмотреть, что содержится в el, попробуйте вывести на экран в отдельной ячейке places_voskresensk.loc[1] (или другое число))


In [72]:
places_voskresensk.loc[1]

id                                1902
region           Нижегородская область
municipality       Воскресенский район
settlement                    Докукино
type                                 с
population                         237
children                            40
latitude_dms                  56.43.39
longitude_dms                045.41.59
latitude_dd                    56.7275
longitude_dd                   45.6997
oktmo                      2.26224e+10
dadata                               0
rosstat                              1
Name: 1, dtype: object

In [31]:
places_by_oktmo['22622416166'] # пример того что выдает словарь по ключу - коду октмо

id                                4649
region           Нижегородская область
municipality       Воскресенский район
settlement                   Рассадино
type                                 д
population                          11
children                             2
latitude_dms                  56.51.18
longitude_dms                045.05.22
latitude_dd                     56.855
longitude_dd                   45.0894
oktmo                      22622416166
dadata                               0
rosstat                              1
Name: 5, dtype: object

In [ ]:
# переходим к датафрейму со школами 
# сначала меняем тип данных в столбцах с широтой и долгой — были строки, а нужны числа (float)

In [87]:
schools_nn['geo_lat'].dtype

dtype('O')

In [33]:
schools_nn['geo_lat'] = schools_nn['geo_lat'].astype('float')
schools_nn['geo_long'] = schools_nn['geo_long'].astype('float')
print(schools_nn['geo_lat'].dtype)
print(schools_nn['geo_long'].dtype)

float64
float64


In [26]:
# удаляем дубликаты 
schools_nn[schools_nn['url'].duplicated()]

,geo_lat,geo_long,url,text
317,55.855785,45.458679,https://schoolotzyv.ru/component/schools/9-rus...,Высокоосельская школа
405,56.196832,43.855258,https://schoolotzyv.ru/component/schools/9-rus...,"Школа №88 ""Новинская"" Нижний Новгород"
814,54.916123,43.347369,https://schoolotzyv.ru/component/schools/9-rus...,Лицей №3 Саров
824,54.916123,43.347369,https://schoolotzyv.ru/component/schools/9-rus...,Лицей №15 Саров
850,56.235430,43.415811,https://schoolotzyv.ru/component/schools/9-rus...,Школа №23 Дзержинск
924,56.335550,44.113685,https://schoolotzyv.ru/component/schools/9-rus...,Школа №10 Бор
989,56.326388,43.955887,https://schoolotzyv.ru/component/schools/9-rus...,Гимназия №2 Нижний Новгород
996,56.289056,43.851548,https://schoolotzyv.ru/component/schools/9-rus...,Школа №75 Нижний Новгород
999,56.312565,43.864385,https://schoolotzyv.ru/component/schools/9-rus...,Школа №109 Нижний Новгород
1004,56.302752,43.880537,https://schoolotzyv.ru/component/schools/9-rus...,Школа №167 Нижний Новгород


In [35]:
schools_nn = schools_nn[~schools_nn['url'].duplicated()]
schools_nn = schools_nn.reset_index(drop = True)

In [ ]:
# собираем такой же словарь, как с населенными пунктами, только ключом будет url школы 

In [36]:
schools_by_url = {}
for i in range(len(schools_nn)):
    el = schools_nn.loc[i]
    schools_by_url[el['url']] = el
len(schools_by_url)

1222

### 5. Расчет матрицы расстояний

Переходим к расчету расстояний между каждым населенным пунктом и каждой школой (матрица расстояний). За основу взят туториал Dana Lindquist https://medium.com/@danalindquist/finding-the-distance-between-two-lists-of-geographic-coordinates-9ace7e43bb2f

In [37]:
# переводим координаты из градусов в радианы, потому что математические формулы обычно требуют 
# значения в радианах, а не в градусах.
# делаем это с помощью библиотеки numpy

schools_nn[['lat_radians_A','long_radians_A']] = (
    np.radians(schools_nn.loc[:,['geo_lat','geo_long']])
)
places_voskresensk[['lat_radians_B','long_radians_B']] = (
    np.radians(places_voskresensk.loc[:,['latitude_dd','longitude_dd']])
)

In [38]:
# считаем расстояния с помощью формулы из библиотеки scikit-learn
# она вычисляет гаверсинусное расстояние, то есть представляет форму Земли как идеальную сферу (а не геоид, 
# как на самом деле), и за счет этого обеспечивает быстрые вычисления
# если требуется измерение в км, то нужно умножить на 6371, если в милях — на 3959


dist = sklearn.neighbors.DistanceMetric.get_metric('haversine')
dist_matrix = (dist.pairwise
    (schools_nn[['lat_radians_A','long_radians_A']],
     places_voskresensk[['lat_radians_B','long_radians_B']])*6371
)

In [40]:
# создаем матрицу расстояний — таблицу, в которой индексами будут url школ,
# колонками - октмо населенных пунктов, а в ячейках будет расстояние от каждого населенного пункта до каждой школы

df_dist_matrix = (
    pd.DataFrame(dist_matrix,index=schools_nn['url'], 
                 columns=places_voskresensk['oktmo'])
)

In [41]:
df_dist_matrix

oktmo,22622404111,22622408141,22622440121,22622440151,22622420111,22622416166,22622420181,22622460141,22622460111,22622404126,...,22622416131,22622452106,22622424161,22622432101,22622456121,22622456126,22622460136,22622416106,22622408221,22622416176
url,,,,,,,,,,,,,,,,,,,,,
https://schoolotzyv.ru/component/schools/9-russia/146-nizhegorodskaya/72980-bolshemurashkinskaya-shkola,127.780010,119.201251,121.505056,121.558613,138.361254,120.534432,148.722942,132.194424,142.803590,135.330889,...,121.458973,117.701803,132.833291,103.604210,92.130892,96.823597,134.277545,120.449068,122.370188,116.026382
https://schoolotzyv.ru/component/schools/9-russia/146-nizhegorodskaya/72981-xolyazinskaya-shkola,123.720058,115.823127,117.546010,117.649458,134.471863,116.243516,144.827854,128.204140,138.759703,131.170195,...,117.191972,113.765678,129.002524,100.337598,88.381831,93.128736,130.258931,116.184689,119.259356,111.681204
https://schoolotzyv.ru/component/schools/9-russia/146-nizhegorodskaya/72982-kishkinskaya-shkola,118.412264,112.124641,112.501179,112.726399,129.565667,110.337659,139.892652,123.062810,133.469609,125.592787,...,111.350746,108.785140,124.245367,96.933304,83.931064,88.787344,125.044115,110.352227,116.124489,105.629020
https://schoolotzyv.ru/component/schools/9-russia/146-nizhegorodskaya/72983-sovetskaya-shkola,125.547781,118.591238,119.547172,119.728550,136.566682,117.629897,146.906802,130.145235,140.603053,132.808803,...,118.629160,115.806887,131.188492,103.240912,90.730090,95.542290,132.151483,117.628477,122.298962,112.948994
https://schoolotzyv.ru/component/schools/9-russia/146-nizhegorodskaya/72984-karabatovskaya-shkola,128.138590,123.418979,122.566730,122.928302,139.735407,119.245118,150.004291,132.979666,143.171487,134.932790,...,120.355214,118.937253,134.590252,108.458941,94.722713,99.652593,134.865274,119.374763,127.767941,114.319733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
https://schoolotzyv.ru/component/schools/9-russia/146-nizhegorodskaya/74321-morenovskaya-sanatorno-lesnaya-shkola,109.968831,116.800653,106.948276,108.149707,123.425079,97.338667,132.673613,115.571593,123.769819,114.042900,...,98.851499,104.211814,119.796793,105.470310,87.379967,92.138507,116.737566,98.057401,124.171646,91.788609
https://schoolotzyv.ru/component/schools/9-russia/146-nizhegorodskaya/74322-shkola-internat,104.968841,114.928398,102.785402,104.197396,118.640925,91.580269,127.413489,110.651597,118.089019,108.138781,...,93.169509,100.396198,115.530698,104.960729,86.403053,90.886306,111.579893,92.457350,122.939105,86.021124
https://schoolotzyv.ru/component/schools/9-russia/146-nizhegorodskaya/74323-shkola-internat,120.463279,138.754476,121.059528,122.986539,133.820130,105.852909,140.460314,125.955933,130.299063,120.332582,...,107.527701,119.887320,132.554991,132.654529,114.263136,117.778936,126.038017,107.163618,147.985108,100.901045


In [42]:
# нам нужно расстояние от населенных пунктов до школ, а не наоборот, поэтому транспонируем таблицу —
# меняем индексы и колонки местами


In [43]:


df_dist_matrix = df_dist_matrix.T
df_dist_matrix

url,https://schoolotzyv.ru/component/schools/9-russia/146-nizhegorodskaya/72980-bolshemurashkinskaya-shkola,https://schoolotzyv.ru/component/schools/9-russia/146-nizhegorodskaya/72981-xolyazinskaya-shkola,https://schoolotzyv.ru/component/schools/9-russia/146-nizhegorodskaya/72982-kishkinskaya-shkola,https://schoolotzyv.ru/component/schools/9-russia/146-nizhegorodskaya/72983-sovetskaya-shkola,https://schoolotzyv.ru/component/schools/9-russia/146-nizhegorodskaya/72984-karabatovskaya-shkola,https://schoolotzyv.ru/component/schools/9-russia/146-nizhegorodskaya/72985-kurlakovskaya-shkola,https://schoolotzyv.ru/component/schools/9-russia/146-nizhegorodskaya/72986-ivanovskaya-shkola,https://schoolotzyv.ru/component/schools/9-russia/146-nizhegorodskaya/72987-grigorovskaya-shkola,https://schoolotzyv.ru/component/schools/9-russia/146-nizhegorodskaya/72988-rozhdestvenskaya-shkola,https://schoolotzyv.ru/component/schools/9-russia/146-nizhegorodskaya/72990-bolshemurashkinskaya-shkola-internat-8-vida,...,https://schoolotzyv.ru/component/schools/9-russia/146-nizhegorodskaya/74316-gou-zolinskaya-shkola-internat,https://schoolotzyv.ru/component/schools/9-russia/146-nizhegorodskaya/74317-gou-popovskaya-shkola-internat,https://schoolotzyv.ru/component/schools/9-russia/146-nizhegorodskaya/74318-gou-bolshemurashkinskaya-shkola-internat-2-vida,https://schoolotzyv.ru/component/schools/9-russia/146-nizhegorodskaya/74319-gou-gorbatovskaya-shkola-internat-1-vida,https://schoolotzyv.ru/component/schools/9-russia/146-nizhegorodskaya/74320-licej-internat,https://schoolotzyv.ru/component/schools/9-russia/146-nizhegorodskaya/74321-morenovskaya-sanatorno-lesnaya-shkola,https://schoolotzyv.ru/component/schools/9-russia/146-nizhegorodskaya/74322-shkola-internat,https://schoolotzyv.ru/component/schools/9-russia/146-nizhegorodskaya/74323-shkola-internat,https://schoolotzyv.ru/component/schools/9-russia/146-nizhegorodskaya/58100-shkola-131,https://schoolotzyv.ru/component/schools/9-russia/146-nizhegorodskaya/119073-soromovskaya-gimnaziya
oktmo,,,,,,,,,,,,,,,,,,,,,
22622404111,127.780010,123.720058,118.412264,125.547781,128.138590,125.811316,113.969575,130.605899,135.766169,128.110032,...,158.254617,592.453221,128.112606,161.816442,107.448775,109.968831,104.968841,120.463279,106.284631,109.033181
22622408141,119.201251,115.823127,112.124641,118.591238,123.418979,119.756405,105.110305,124.029928,126.293517,119.669620,...,173.713819,562.871841,119.583388,174.645328,119.788583,116.800653,114.928398,138.754476,115.547045,122.118390
22622440121,121.505056,117.546010,112.501179,119.547172,122.566730,119.976315,107.590143,124.694483,129.369383,121.859827,...,157.993814,585.299954,121.847301,160.731404,105.993029,106.948276,102.785402,121.059528,103.909753,107.818924
22622440151,121.558613,117.649458,112.726399,119.728550,122.928302,120.229849,107.601107,124.912344,129.359699,121.925008,...,159.771042,583.279633,121.905291,162.353264,107.563081,108.149707,104.197396,122.986539,105.275720,109.437242
22622420111,138.361254,134.471863,129.565667,136.566682,139.735407,137.066960,124.383904,141.751865,146.125952,138.732792,...,171.855176,589.431777,138.709916,175.545515,121.190330,123.425079,118.640925,133.820130,119.920841,122.773815
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22622456126,96.823597,93.128736,88.787344,95.542290,99.652593,96.390499,82.754205,100.874301,104.374345,97.233883,...,150.976500,566.898346,97.186256,150.967872,96.391696,92.138507,90.886306,117.778936,91.331990,98.952228
22622460136,134.277545,130.258931,125.044115,132.151483,134.865274,132.468169,120.416777,137.240142,142.209125,134.618298,...,164.183923,593.557299,134.614389,168.065962,113.862255,116.737566,111.579893,126.038017,112.937576,115.368898
22622416106,120.449068,116.184689,110.352227,117.628477,119.374763,117.555233,106.975473,122.476226,128.679408,120.720407,...,144.980027,598.368248,120.757552,148.720883,94.543092,98.057401,92.457350,107.163618

### 6. Выбор ближайшего объекта

Выбираем ближайшую школу к каждому населенному пункту.

In [44]:
# дальше мы сравниваем каждое значение в строке с предыдущим, чтобы проверить, больше оно или меньше. Меньшее записываем в переменную

schools_and_min_value_by_oktmo = {} # пустой словарь, где ключом будет код октмо, 
                                    #значениями — минимальное расстояние

columns = df_dist_matrix.columns # список с названиями колонок (url школ)
for current_oktmo in df_dist_matrix.index:
    distances = df_dist_matrix.loc[current_oktmo]
    min_distance = None # нужно для начала отсчета, чтобы было с чем сравнивать первое значение в строке
    min_j = None
    for j in range(len(columns)):
        if min_distance is None or min_distance > distances[j]: # сравниваем каждое значение в строке с 
            # предыдущим, чтобы проверить, больше оно или меньше. Меньшее записываем в переменную
            min_distance = distances[j]
            min_j = j 
    schools_and_min_value_by_oktmo[current_oktmo] = (columns[min_j], min_distance)
# заполняем словарь: ключ — код октмо, значение — url ближайшей школы и расстояние до нее
len(schools_and_min_value_by_oktmo)

158

In [45]:
schools_and_min_value_by_oktmo['22622440151'] # пример того, что выдает словарь по коду октмо

('https://schoolotzyv.ru/component/schools/9-russia/146-nizhegorodskaya/73604-chuxlomskaya-shkola',
 0.014360770987876591)

In [50]:
# добавляем колонки с мин расстоянием и url школы в датафрейм places_voskresensk (с населенными пунктами)
# создаем списки с полученными выше url и км, чтобы затем добавить их к датафрейму с населенными пунктами

school_url_column = [] 
min_distance_column = []
for oktmo in places_voskresensk['oktmo']:
    (url, distns) = schools_and_min_value_by_oktmo[oktmo]
    school_url_column.append(url) # берем из созданного выше словаря url ближайшей школы(url) и км до нее (distns),
    # добавляем их в словари в том порядке, в каком соответствующие им коды октмо расположены в датафрейме places_voskresensk
    min_distance_column.append(distns)
places_voskresensk['school_url_column'] = school_url_column # добавляем новую колонку с url ближайших школ
places_voskresensk['min_distance_column'] = min_distance_column # добавляем новую колонку с км до этих школ
len(places_voskresensk)

158

In [46]:
places_voskresensk

,id,region,municipality,settlement,type,population,children,latitude_dms,longitude_dms,latitude_dd,longitude_dd,oktmo,dadata,rosstat,lat_radians_B,long_radians_B
0,6,Нижегородская область,Воскресенский район,Апариха,д,0,0,56.53.40,045.19.08,56.894444,45.318889,22622404111,0,1,0.992995,0.790964
1,1902,Нижегородская область,Воскресенский район,Докукино,с,237,40,56.43.39,045.41.59,56.727500,45.699722,22622408141,0,1,0.990082,0.797611
2,2731,Нижегородская область,Воскресенский район,Капустиха,д,215,41,56.49.41,045.21.58,56.828056,45.366111,22622440121,0,1,0.991837,0.791788
3,2867,Нижегородская область,Воскресенский район,Чухломка,д,187,50,56.49.20,045.24.06,56.822222,45.401667,22622440151,0,1,0.991735,0.792409
4,4089,Нижегородская область,Воскресенский район,Большие Отары,д,362,79,56.57.43,045.30.31,56.961944,45.508611,22622420111,0,1,0.994173,0.794275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,153214,Нижегородская область,Воскресенский район,Клюкино,д,29,2,56.35.22,045.22.43,56.589444,45.378611,22622456126,0,1,0.987672,0.792006
154,153490,Нижегородская область,Воскресенский район,Раскаты,д,238,22,56.56.43,045.22.59,56.945278,45.383056,22622460136,0,1,0.993883,0.792084
155,154465,Нижегородская область,Воскресенский район,Александровка,д,0,0,56.51.07,045.06.51,56.851944,45.114167,22622416106,0,1,0.992254,0.787391
156,154802,Нижегородская область,Воскресенский район,Успенское,с,1,0,56.42.32,045.51.24,56.708889,45.856667,22622408221,0,1,0.989757,0.800350


In [52]:
# из словаря со школами достаем названия школ по url и тоже добавляем их к датафрейму places_voskresensk новой колонкой


school_name = []

for url in places_voskresensk['school_url_column']:
    school_name.append(schools_by_url[url]['text'])
places_voskresensk['school_name'] = school_name

In [54]:
# в конце появились 3 новые колонки — url ближайшей школы, сколько до нее км, название школы

places_voskresensk

,id,region,municipality,settlement,type,population,children,latitude_dms,longitude_dms,latitude_dd,longitude_dd,oktmo,dadata,rosstat,lat_radians_B,long_radians_B,school_url_column,min_distance_column,school_name
0,6,Нижегородская область,Воскресенский район,Апариха,д,0,0,56.53.40,045.19.08,56.894444,45.318889,22622404111,0,1,0.992995,0.790964,https://schoolotzyv.ru/component/schools/9-rus...,3.330156,Благовещенская школа
1,1902,Нижегородская область,Воскресенский район,Докукино,с,237,40,56.43.39,045.41.59,56.727500,45.699722,22622408141,0,1,0.990082,0.797611,https://schoolotzyv.ru/component/schools/9-rus...,0.009381,Докукинская школа
2,2731,Нижегородская область,Воскресенский район,Капустиха,д,215,41,56.49.41,045.21.58,56.828056,45.366111,22622440121,0,1,0.991837,0.791788,https://schoolotzyv.ru/component/schools/9-rus...,2.248417,Чухломская школа
3,2867,Нижегородская область,Воскресенский район,Чухломка,д,187,50,56.49.20,045.24.06,56.822222,45.401667,22622440151,0,1,0.991735,0.792409,https://schoolotzyv.ru/component/schools/9-rus...,0.014361,Чухломская школа
4,4089,Нижегородская область,Воскресенский район,Большие Отары,д,362,79,56.57.43,045.30.31,56.961944,45.508611,22622420111,0,1,0.994173,0.794275,https://schoolotzyv.ru/component/schools/9-rus...,0.013753,Большеотарская школа
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,153214,Нижегородская область,Воскресенский район,Клюкино,д,29,2,56.35.22,045.22.43,56.589444,45.378611,22622456126,0,1,0.987672,0.792006,https://schoolotzyv.ru/component/schools/9-rus...,4.132106,Нестиарская школа
154,153490,Нижегородская область,Воскресенский район,Раскаты,д,238,22,56.56.43,045.22.59,56.945278,45.383056,22622460136,0,1,0.993883,0.792084,https://schoolotzyv.ru/component/schools/9-rus...,0.009641,Раскатская школа
155,154465,Нижегородская область,Воскресенский район,Александровка,д,0,0,56.51.07,045.06.51,56.851944,45.114167,22622416106,0,1,0.992254,0.787391,https://schoolotzyv.ru/component/schools/9-rus...,3.524948,Владимирская школа
156,154802,Нижегородская область,Воскресенский район,Успенское,с,1,0,56.42.32,045.51.24,56.708889,45.856667,22622408221,0,1,0.989757,0.800350,https://schoolotzyv.ru/component/schools/9-rus...,9.567901,Красноярская школа


In [55]:
# сохраним полученный результат в csv 


places_voskresensk.to_csv('voskresensk_closest_schools.csv')

### 7. Анализ данных

In [56]:
# Дальше можно анализировать данные.
# Например, выбрать 15 наиболе удаленных от школ объектов методов nlarest
# подробнее об nlargest и nsmallest https://istories.media/workshops/2021/03/19/python-biblioteka-pandas-chast-3/ 

In [57]:
places_voskresensk.nlargest(15,'min_distance_column')

,id,region,municipality,settlement,type,population,children,latitude_dms,longitude_dms,latitude_dd,longitude_dd,oktmo,dadata,rosstat,lat_radians_B,long_radians_B,school_url_column,min_distance_column,school_name
41,41506,Нижегородская область,Воскресенский район,Красная Звезда,д,0,0,56.29.48,045.33.09,56.496667,45.552500,22622432126,0,1,0.986053,0.795041,https://schoolotzyv.ru/component/schools/9-rus...,11.260120,Егоровская школа
22,25057,Нижегородская область,Воскресенский район,Автулиха,д,10,0,56.37.54,045.27.45,56.631667,45.462500,22622456106,0,1,0.988409,0.793470,https://schoolotzyv.ru/component/schools/9-rus...,10.768442,Егоровская школа
85,83050,Нижегородская область,Воскресенский район,Томилиха,д,2,0,56.41.42,045.51.14,56.695000,45.853889,22622408206,0,1,0.989514,0.800301,https://schoolotzyv.ru/component/schools/9-rus...,10.087465,Докукинская школа
141,138624,Нижегородская область,Воскресенский район,Бахарево,д,22,2,56.41.55,045.51.08,56.698611,45.852222,22622408111,0,1,0.989577,0.800272,https://schoolotzyv.ru/component/schools/9-rus...,9.853790,Докукинская школа
6,4922,Нижегородская область,Воскресенский район,Петрово,д,0,0,57.03.02,045.33.42,57.050556,45.561667,22622420181,0,1,0.995720,0.795201,https://schoolotzyv.ru/component/schools/9-rus...,9.822884,Староустинская школа
19,21738,Нижегородская область,Воскресенский район,Красные Поляны,д,1,0,56.50.53,045.46.25,56.848056,45.773611,22622424141,0,1,0.992186,0.798900,https://schoolotzyv.ru/component/schools/9-rus...,9.708871,Красноярская школа
156,154802,Нижегородская область,Воскресенский район,Успенское,с,1,0,56.42.32,045.51.24,56.708889,45.856667,22622408221,0,1,0.989757,0.800350,https://schoolotzyv.ru/component/schools/9-rus...,9.567901,Красноярская школа
62,58690,Нижегородская область,Воскресенский район,Завод,д,15,0,56.37.41,045.25.02,56.628056,45.417222,22622456116,0,1,0.988346,0.792680,https://schoolotzyv.ru/component/schools/9-rus...,8.986548,Нестиарская школа
27,27980,Нижегородская область,Воскресенский район,Шурговаш,с,114,12,56.46.25,045.14.10,56.773611,45.236111,22622416186,1,0,0.990886,0.789519,https://schoolotzyv.ru/component/schools/9-rus...,8.894897,Владимирская школа
65,60613,Нижегородская область,Воскресенский район,Прудовские,д,3,0,57.03.11,045.31.26,57.053056,45.523889,22622420186,0,1,0.995764,0.794542,https://schoolotzyv.ru/component/schools/9-rus...,8.605541,Староустинская школа


In [60]:
# или посчитать, сколько детей в Воскресенском районе Нижегородской обл. живут дальше 5 км от школы

km5 = places_voskresensk[places_voskresensk['min_distance_column'] > 5]
print(places_voskresensk['children'].sum()) # всего детей
print(km5['children'].sum()) # живут дальше 5 км

3385
269


In [61]:
# или среднее расстояние от населенного пункта до школы 

places_voskresensk['min_distance_column'].mean()

3.91580963122014

Готово, спасибо, что дошли до конца!